# TODOs

* PCA to see which feature has how much influence
* implement scoring function, eating food gives plus, getting smaller distance to food gives plus, reducing distance to enemy wolf gives minus etc.?

In [ ]:
import pandas as pd
from sklearn import tree, naive_bayes, svm
import ast
import math
import pickle
import os
import glob

Make all the constants from the game available here to use.

In [ ]:
#[General_Constants]
FIELD_WIDTH = 19
FIELD_HEIGHT = 15

#[Game_Constants]
NO_ITERATIONS = 100
MAX_CALC_TIME = 100000

#[Field_Constants]
CELL_EMPTY = '.'
CELL_SHEEP_1 = 'S'
CELL_SHEEP_1_d = 'U'
CELL_WOLF_1 = 'W'
CELL_SHEEP_2 = 's'
CELL_SHEEP_2_d = 'u'
CELL_WOLF_2 = 'w'
CELL_GRASS = 'g'
CELL_RHUBARB = 'r'
CELL_FENCE = '#'


#[Movements]
MOVE_NONE = 0
MOVE_UP = -1
MOVE_DOWN = 1
MOVE_LEFT = -2
MOVE_RIGHT = 2

#[Awards]
AWARD_RHUBARB = 5
AWARD_GRASS = 1

# Load data

Load all games into one dataframe

In [ ]:
path = "../training_data/subset1" 
all_files = glob.glob(path + "/*.csv")   

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

#preview the final 5 lines
training_data[-1].head()

# Feature selection and Instance selection 

### General Note: 
I imported the class of my Python file to use the same functions and prevent duplicating code. I have to use the same feature preparation in the training phase as in the prediction pmhase. Thus all feature engineering is done in the python file (this also allows for easier debugging).

## Sheep

### Feature Selection:
My baseline was the Search Algorithm from assignment 1. I used the movement determined by the search algorithm as a feature for my sheep. I then tried multiple features and looked at their correlation to the actual move made. I dropped features with poor correlation. Thus, I ended up with taking the player into consideration, as player 1 always starts in the top, player 2 on the bottom. Also, the direction of the enemy sheep has an influence, probably favoring an agressive playstile.

### Data Selection
I used all rows from the winning side performing a sheep-move. I used different configurations of agents from assignment 1 and ran them on random maps to scrape the results. I didn't use the provided data, as I didn't know the quality of the games, as the second worst agents still wins against the worst agent resulting it to be a "win" although both agents are in fact not optimal. I also ignored all games that ended because of an exception.

In [ ]:
from importlib import reload  # to get changes in code
import chriweb_a2
chriweb_a2 = reload(chriweb_a2)
ii = chriweb_a2.IntrepidIbex()

X_sheep = []
Y_sheep = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue
        
    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 sheep','player2 sheep'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
                
        game_features = ii.get_features_sheep(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_sheep.append(game_features)
        Y_sheep.append(move)  

In [ ]:
df_investigate_s = pd.DataFrame(X_sheep)
df_investigate_s['y'] = Y_sheep
df_investigate_s

See correlation of each feature to result. I dropped features that had not enough correlation.

In [ ]:
df_investigate_s.corr()['y']

## Wolf

Explain here in text which feature you used for the wolf, and which data you used for training.

### Feature Selection:
My baseline was the Search Algorithm from assignment 1. I used the movement determined by the search algorithm as a feature for my wolf. I then tried multiple features and looked at their correlation to the actual move made. I dropped features with poor correlation. Thus, I ended up with the following features:
* Player 1 or 2, as player 1 always starts in the top, player 2 on the bottom. 
* Direction needed to get to the enemy sheep (as the wolf needs to eat it)
* Get the degrees of freedom of neighboring fields (reasoning could be to cut of enemy sheep?)

### Data Selection
I used all rows from the winning side performing a wolf-move. I used different configurations of agents from assignment 1 and ran them on random maps to scrape the results. I didn't use the provided data, as I didn't know the quality of the games, as the second worst agents still wins against the worst agent resulting it to be a "win" although both agents are in fact not optimal. I also ignored all games that ended because of an exception.

In [ ]:
from importlib import reload  # to get changes in code
import chriweb_a2
chriweb_a2 = reload(chriweb_a2)
ii = chriweb_a2.IntrepidIbex()

X_wolf = []
Y_wolf = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue

    rhubarb = 'r'
    grass = 'g'

    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 wolf','player2 wolf'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
        
        game_features = ii.get_features_wolf(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_wolf.append(game_features)
        Y_wolf.append(move)  

In [ ]:
df_investigate_w = pd.DataFrame(X_wolf)
df_investigate_w['y'] = Y_wolf
df_investigate_w

In [ ]:
df_investigate_w.corr()['y']

# Build Sheep Model

Split into train and test data to measure performance

In [ ]:
from sklearn import model_selection
x_train_s, x_test_s, y_train_s, y_test_s = model_selection.train_test_split(X_sheep, Y_sheep, test_size=0.25)

## Train & Test sheep

Allowed algorithms:
* Naive Bayes (GaussianNB, MultinominalNB, ComplementNB, BernoulliNB)
* Decision Tree
* Support Vector Machine (SVC, NuSVC, LinearSVC)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

## Naive Bayes

In [ ]:
# sheep_NB = naive_bayes.GaussianNB()
# # sheep_NB = naive_bayes.MultinomialNB()
# # sheep_NB = naive_bayes.ComplementNB()
# # sheep_NB = naive_bayes.BernoulliNB()
# sheep_NB = sheep_NB.fit(x_train_s,y_train_s)
# y_pred_s = sheep_NB.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

## Decision Tree

In [ ]:
# sheep_tree = tree.DecisionTreeClassifier()
# sheep_tree = sheep_tree.fit(x_train_s,y_train_s)
# # sheep_tree = sheep_tree.fit(np.array(x_train_s).reshape(-1,1),y_train_s)
# # y_pred_s = sheep_tree.predict(np.array(x_test_s).reshape(-1,1))
# y_pred_s = sheep_tree.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

## SVM

The precision/recall is similar to the decision tree, but I choose SVM as it performed better when run against my agent from assignment 1.

In [ ]:
sheep_svm = svm.SVC()
sheep_svm = sheep_svm.fit(x_train_s,y_train_s)
y_pred_s = sheep_svm.predict(x_test_s)

cm = confusion_matrix(y_test_s, y_pred_s.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

# Train wolf

## Decision Tree

In [ ]:
from sklearn import model_selection
x_train_w, x_test_w, y_train_w, y_test_w = model_selection.train_test_split(X_wolf, Y_wolf, test_size=0.25)

In [ ]:
wolf_tree = tree.DecisionTreeClassifier()
wolf_tree = wolf_tree.fit(x_train_w,y_train_w)
y_pred_w = wolf_tree.predict(x_test_w)

cm = confusion_matrix(y_test_w, y_pred_w.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

# Save models to files


Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [ ]:
sheep_filename = 'chriweb_sheep_model.sav'
wolf_filename = 'chriweb_wolf_model.sav'

pickle.dump(sheep_svm,open(sheep_filename,'wb'))
pickle.dump(wolf_tree,open(wolf_filename,'wb'))